In [45]:
%%writefile testUtility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re

def readFile(filepath):
  with open(filepath, 'r') as stream:
            try:
              return yaml.safe_load(stream)
            except yaml.YAMLError as exc:
              logging.error(exc)
             
def replacer(string, char):
  pattern = char + '{2,}'
  string = re.sub(pattern, char, string)
  return string
            
def headerValidation(df, table_confic):
  df.columns = df.columns.str.lower()
  df.columns = df.columns.str.replace('[^\w]','_',regex=TRUE)
  df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
  df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
  expected_col = list(map(lambda x: x.lower(), table_config['columns']))
  expected_col.sort()
  df.columns = list(map(lambda x: x.lower(), list(df.columns)))
  df = df.reindex(sorted(df.columns), axis = 1)
  if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("coloumn name & length validation passed")
        return 1
  else:
        print("column name & length validation failed")
        mismatchedColumnsFile = list(set(df.columns).difference(expected_col))
        print("The following columns are not in the YAML file", mismatchedColumnsFile)
        missingYAMLFile = list(set(expected_col).difference(df.columns))
        print("The following YAML columns are not in the file", missingYAMLFile)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testUtility.py


In [46]:
%%writefile shoppers.yaml
file_type: csv 
dataset_name: acquire-valued-shoppers-challenge
file_name: transactions
table_nameL: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
  - offer
  - category
  - quantity
  - company
  - offervalue
  - brand

Overwriting shoppers.yaml


In [ ]:
import sys
!{sys.executable} -m pip install --user yaml

In [ ]:
import testUtility as util
configData = util.readFile("shoppers.yaml")

In [ ]:
configData['inbound_delimiter']

In [ ]:
configData

In [ ]:
import sys
!{sys.executable} -m pip install --user pandas as pd

In [ ]:
import pandas as pd
df_sample = pd.read_csv("transactions.csv", delimiter = ',')
df_sample.head()

In [ ]:
fileType = configData['fileType']
sourceFile = "./" + configData['fileName'] + f'.{fileType}'
df = pd.read_csv(sourceFile, configData['inbound_delimiter'])
df.head()

In [ ]:
util.colHeaderVal(df,configData)

In [ ]:
print("columns of file are:" ,df.columns)
print("columns of YAML are:" ,configData['columns'])

In [ ]:
if util.colHeaderVal(df,configData) == 0:
    print("validation failed")

else:
    print("validation passed")

In [ ]:
pd.read_csv("./transactions.csv")